In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from typing import Dict, Any, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# ========== PREPARAÇÃO DE DADOS ==========

def get_clean_data(df: pd.DataFrame, target_col: str = "Vazao_BBR") -> pd.DataFrame:
    """
    Remove apenas linhas com valores inválidos (-1) no target.
    """
    df_clean = df[df[target_col] != -1].copy().reset_index(drop=True)
    print(f"  [DADOS LIMPOS] {len(df_clean)} amostras válidas (removidos {len(df) - len(df_clean)} com -1)")
    return df_clean

def apply_random_mask(df: pd.DataFrame, missing_fraction: float, seed: int = None) -> pd.DataFrame:
    """
    Aplica máscara aleatória para BASELINES - marca valores a serem 'escondidos' para teste.
    """
    df_masked = df.copy()
    n_samples = len(df_masked)
    n_mask = max(1, int(missing_fraction * n_samples))
    
    if seed is not None:
        np.random.seed(seed)
    
    mask_indices = np.random.choice(df_masked.index, size=n_mask, replace=False)
    df_masked['mask_applied'] = 0
    df_masked.loc[mask_indices, 'mask_applied'] = 1
    
    print(f"    Máscara aplicada: {n_mask}/{n_samples} amostras ({missing_fraction*100:.0f}%)")
    return df_masked

# ========== FEATURE ENGINEERING ==========

def engineer_features_for_imputation(df: pd.DataFrame, target_col: str = "Vazao_BBR") -> pd.DataFrame:
    """
    Feature engineering SEM data leakage para imputação.
    Regras:
    - Nunca usar o valor ATUAL do target para criar features.
    - Substitui inf/NaN por valores numéricos seguros.
    """
    df = df.copy()
    
    # ✅ Garantir que Data está como datetime e criar features temporais básicas
    if 'Data' in df.columns:
        df['Data'] = pd.to_datetime(df['Data'])
        df['hour'] = df['Data'].dt.hour
        df['day_of_week'] = df['Data'].dt.dayofweek
        df['day_of_month'] = df['Data'].dt.day

    # ✅ Features derivadas de outras colunas (não do target)
    df["Atraso_log"] = np.log1p(df["Atraso(ms)"].clip(lower=0))
    df["Hop_inv"] = 1 / (df["Hop_count"] + 1)
    df["Atraso_x_Hop"] = df["Atraso(ms)"] * df["Hop_count"]
    df["Atraso_sq"] = df["Atraso(ms)"] ** 2
    df["Hop_sq"] = df["Hop_count"] ** 2

    if 'hour' in df.columns:
        df["Atraso_x_hour"] = df["Atraso(ms)"] * df["hour"]
        df["Hop_x_hour"] = df["Hop_count"] * df["hour"]
        df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
        df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)

    # ✅ Features do target baseadas em valores passados
    valid_mask = df[target_col] != -1
    target_series = df[target_col].copy()
    target_series[~valid_mask] = np.nan

    # Lags
    for lag in [1, 2, 3, 6]:
        df[f"Vazao_lag{lag}"] = target_series.shift(lag)

    # Diferenças e variações percentuais
    df["Vazao_diff1"] = target_series.diff(1)
    df["Vazao_diff2"] = target_series.diff(2)
    df["Vazao_pct_change"] = target_series.pct_change()

    # Rolling features (sem vazamento)
    for w in [3, 6]:
        shifted = target_series.shift(1)
        df[f"Vazao_roll_mean_{w}"] = shifted.rolling(window=w, min_periods=w).mean()
        df[f"Vazao_roll_std_{w}"] = shifted.rolling(window=w, min_periods=w).std()
    df["Vazao_roll_max_6"] = shifted.rolling(window=6, min_periods=6).max()
    df["Vazao_roll_min_6"] = shifted.rolling(window=6, min_periods=6).min()

    # Relações derivadas
    lag1 = target_series.shift(1)
    df["Vazao_lag1_div_Atraso"] = lag1 / (df["Atraso(ms)"] + 1)
    df["Vazao_lag1_div_Hops"] = lag1 / (df["Hop_count"] + 1)
    df["Efficiency_lag1"] = lag1 / ((df["Atraso(ms)"] + 1) * (df["Hop_count"] + 1))

    # Transformações seguras
    df["Vazao_lag1_log"] = np.log1p(lag1.clip(lower=0))
    df["Vazao_lag1_sqrt"] = np.sqrt(lag1.clip(lower=0))

    # Estatísticas de janela expandida
    df["Vazao_expanding_mean"] = target_series.shift(1).expanding(min_periods=1).mean()
    df["Vazao_expanding_std"] = target_series.shift(1).expanding(min_periods=3).std()

    # Medidas globais do target
    df['Feature_Vazao_bbr_median'] = target_series.median()
    df['Feature_Vazao_bbr_mean'] = target_series.mean()

    # Restaurar valores -1 originais
    df.loc[~valid_mask, target_col] = -1

    # ✅ Tratamento final: substituir infinitos e NaNs
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    for col in df.select_dtypes(include=[np.number]).columns:
        if col == target_col:
            continue
        if df[col].isna().any():
            if any(k in col for k in ['lag', 'roll', 'diff', 'pct', 'expanding']):
                df[col].fillna(0, inplace=True)
            else:
                df[col].fillna(df[col].median(), inplace=True)

    return df


# ========== CÁLCULO DE MÉTRICAS ==========

def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray, prediction_time: float = None) -> Dict[str, Any]:
    """
    Calcula métricas de regressão de forma robusta.
    """
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    
    mask = ~(np.isnan(y_true) | np.isnan(y_pred) | np.isinf(y_true) | np.isinf(y_pred))
    
    if mask.sum() < 2:
        return {"rmse": None, "nrmse": None, "r2": None, "mape": None, "prediction_time_per_sample": None}
    
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    
    try:
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        nrmse = (rmse / (np.mean(y_true) + 1e-8)) * 100
        rmse_normalized = rmse / 1_000_000
        
        r2 = r2_score(y_true, y_pred)
        r2 = r2 if not np.isnan(r2) and np.isfinite(r2) else None
        
        mape = np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-8))) * 100
        
        time_per_sample = None
        if prediction_time is not None and len(y_true) > 0:
            time_per_sample = round((prediction_time / len(y_true)) * 1000, 4)
        
        return {
            "rmse": round(rmse_normalized, 2),
            "nrmse": round(nrmse, 2),
            "r2": r2,
            "mape": round(mape, 2),
            "prediction_time_per_sample": time_per_sample
        }
    except:
        return {"rmse": None, "nrmse": None, "r2": None, "mape": None, "prediction_time_per_sample": None}

# ========== AVALIAÇÃO BASELINES ==========

def evaluate_baselines(df_clean: pd.DataFrame, missing_fraction: float, target_col: str = "Vazao_BBR") -> Dict:
    """
    Avalia métodos baseline.
    """
    print(f"  [BASELINE] Avaliando fração {missing_fraction:.0%}")
    
    df_masked = apply_random_mask(df_clean, missing_fraction, seed=42)
    mask_indices = df_masked[df_masked['mask_applied'] == 1].index
    y_true = df_masked.loc[mask_indices, target_col].values
    
    results = {}
    df_with_nan = df_masked.copy()
    df_with_nan.loc[mask_indices, target_col] = np.nan
    
    # MÉDIA
    try:
        df_mean = df_with_nan.copy()
        mean_value = df_mean[target_col].mean()
        df_mean[target_col] = df_mean[target_col].fillna(mean_value)
        y_pred = df_mean.loc[mask_indices, target_col].values
        results['Mean'] = calculate_metrics(y_true, y_pred)
    except:
        results['Mean'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # MEDIANA
    try:
        df_median = df_with_nan.copy()
        median_value = df_median[target_col].median()
        df_median[target_col] = df_median[target_col].fillna(median_value)
        y_pred = df_median.loc[mask_indices, target_col].values
        results['Median'] = calculate_metrics(y_true, y_pred)
    except:
        results['Median'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # KNN IMPUTER
    try:
        df_knn = df_with_nan.copy()
        imputer = KNNImputer(n_neighbors=min(5, len(df_clean) // 2))
        df_knn[target_col] = imputer.fit_transform(df_knn[[target_col]]).ravel()
        y_pred = df_knn.loc[mask_indices, target_col].values
        results['KNNImputer'] = calculate_metrics(y_true, y_pred)
    except:
        results['KNNImputer'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # FORWARD FILL
    try:
        df_ffill = df_with_nan.copy()
        df_ffill[target_col] = df_ffill[target_col].ffill().bfill()
        y_pred = df_ffill.loc[mask_indices, target_col].values
        results['ForwardFill'] = calculate_metrics(y_true, y_pred)
    except:
        results['ForwardFill'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # BACKWARD FILL
    try:
        df_bfill = df_with_nan.copy()
        df_bfill[target_col] = df_bfill[target_col].bfill().ffill()
        y_pred = df_bfill.loc[mask_indices, target_col].values
        results['BackwardFill'] = calculate_metrics(y_true, y_pred)
    except:
        results['BackwardFill'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # ROLLING MEAN
    try:
        df_rolling = df_with_nan.copy()
        df_rolling[target_col] = df_rolling[target_col].rolling(window=3, min_periods=1).mean()
        df_rolling[target_col] = df_rolling[target_col].ffill().bfill()
        y_pred = df_rolling.loc[mask_indices, target_col].values
        results['RollingMean'] = calculate_metrics(y_true, y_pred)
    except:
        results['RollingMean'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # INTERPOLAÇÃO LINEAR
    try:
        df_linear = df_with_nan.copy()
        df_linear[target_col] = df_linear[target_col].interpolate(method='linear').ffill().bfill()
        y_pred = df_linear.loc[mask_indices, target_col].values
        results['LinearInterpolation'] = calculate_metrics(y_true, y_pred)
    except:
        results['LinearInterpolation'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    return results

# ========== GRID SEARCH PARA MODELOS BASE ==========

def get_optimized_base_models(X_train, y_train, n_jobs=-1):
    """
    Retorna modelos base otimizados com Grid Search.
    """
    print(f"      🔍 Iniciando Grid Search nos modelos base...")
    
    # Configurar CV temporal
    cv_splits = min(3, len(X_train) // 20)
    tscv = TimeSeriesSplit(n_splits=cv_splits)
    
    optimized_models = []
    
    # ===== XGBoost =====
    print(f"        - XGBoost...")
    xgb_params = {
        'n_estimators': [100, 150, 200],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.05, 0.1, 0.15],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9]
    }
    xgb_grid = GridSearchCV(
        XGBRegressor(random_state=42, verbosity=0, n_jobs=1),
        xgb_params,
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=n_jobs,
        verbose=0
    )
    xgb_grid.fit(X_train, y_train)
    optimized_models.append(('xgb', xgb_grid.best_estimator_))
    print(f"          ✓ Melhores params: {xgb_grid.best_params_}")
    
    # ===== Random Forest =====
    print(f"        - Random Forest...")
    rf_params = {
        'n_estimators': [100, 150, 200],
        'max_depth': [10, 15, 20],
        'min_samples_split': [2, 3, 5],
        'min_samples_leaf': [1, 2, 3],
        'max_features': ['sqrt', 'log2']
    }
    rf_grid = GridSearchCV(
        RandomForestRegressor(random_state=42, n_jobs=1),
        rf_params,
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=n_jobs,
        verbose=0
    )
    rf_grid.fit(X_train, y_train)
    optimized_models.append(('rf', rf_grid.best_estimator_))
    print(f"          ✓ Melhores params: {rf_grid.best_params_}")
    
    # ===== Gradient Boosting =====
    print(f"        - Gradient Boosting...")
    gb_params = {
        'n_estimators': [100, 150, 200],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.05, 0.1, 0.15],
        'subsample': [0.7, 0.8, 0.9],
        'min_samples_split': [2, 3, 5]
    }
    gb_grid = GridSearchCV(
        GradientBoostingRegressor(random_state=42),
        gb_params,
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=n_jobs,
        verbose=0
    )
    gb_grid.fit(X_train, y_train)
    optimized_models.append(('gb', gb_grid.best_estimator_))
    print(f"          ✓ Melhores params: {gb_grid.best_params_}")
    
    # ===== KNN =====
    print(f"        - KNN...")
    knn_params = {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'p': [1, 2],
        'metric': ['minkowski', 'euclidean']
    }
    knn_grid = GridSearchCV(
        KNeighborsRegressor(),
        knn_params,
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=n_jobs,
        verbose=0
    )
    knn_grid.fit(X_train, y_train)
    optimized_models.append(('knn', knn_grid.best_estimator_))
    print(f"          ✓ Melhores params: {knn_grid.best_params_}")
    
    # ===== ElasticNet (NOVO) =====
    print(f"        - ElasticNet...")
    en_params = {
        'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],
        'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
        'max_iter': [5000, 10000]
    }
    en_grid = GridSearchCV(
        ElasticNet(random_state=42),
        en_params,
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=n_jobs,
        verbose=0
    )
    en_grid.fit(X_train, y_train)
    optimized_models.append(('elasticnet', en_grid.best_estimator_))
    print(f"          ✓ Melhores params: {en_grid.best_params_}")
    
    return optimized_models

# ===== META-MODELO COM GRID SEARCH =====

def get_optimized_meta_model(X_train, y_train, base_models):
    """
    Otimiza o meta-modelo com Grid Search.
    """
    print(f"      🔍 Otimizando meta-modelo...")
    
    cv_splits = min(3, len(X_train) // 20)
    tscv = TimeSeriesSplit(n_splits=cv_splits)
    
    # Testar Ridge e ElasticNet como meta-modelos
    meta_params = [
        {
            'final_estimator': [Ridge()],
            'final_estimator__alpha': [0.1, 1.0, 10.0, 50.0, 100.0]
        },
        {
            'final_estimator': [ElasticNet(max_iter=10000)],
            'final_estimator__alpha': [0.1, 1.0, 10.0],
            'final_estimator__l1_ratio': [0.1, 0.5, 0.9]
        }
    ]
    
    best_score = float('-inf')
    best_meta = None
    
    for params in meta_params:
        stacking = StackingRegressor(
            estimators=base_models,
            cv=cv_splits,
            n_jobs=-1,
            passthrough=True
        )
        
        grid = GridSearchCV(
            stacking,
            params,
            cv=tscv,
            scoring='neg_mean_squared_error',
            n_jobs=-1,
            verbose=0
        )
        
        grid.fit(X_train, y_train)
        
        if grid.best_score_ > best_score:
            best_score = grid.best_score_
            best_meta = grid.best_estimator_
            best_params = grid.best_params_
    
    print(f"        ✓ Melhor meta-modelo: {best_params}")
    return best_meta

# ========== AVALIAÇÃO STACKING COM GRID SEARCH E CV ==========

def evaluate_stacking_cv(df_clean: pd.DataFrame, test_fraction: float, target_col: str = "Vazao_BBR") -> Dict:
    """
    Avalia Stacking com Grid Search usando Time Series Split.
    """
    import time
    
    print(f"  [STACKING] Avaliando com test_fraction={test_fraction:.0%}")
    
    n_splits = max(3, min(5, int(1 / test_fraction)))
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    all_metrics = []
    all_prediction_times = []
    print(f"    Cross-validation com {n_splits} splits")

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_clean)):
        desired_test_size = int(len(df_clean) * test_fraction)
        if len(test_idx) > desired_test_size:
            test_idx = test_idx[-desired_test_size:]

        df_train = df_clean.iloc[train_idx].copy()
        df_test = df_clean.iloc[test_idx].copy()

        df_train = engineer_features_for_imputation(df_train, target_col)
        df_test = engineer_features_for_imputation(df_test, target_col)

        exclude_cols = {target_col, 'Data', 'mask_applied'}
        feature_cols = [c for c in df_train.columns if c not in exclude_cols]

        df_train = df_train.dropna(subset=feature_cols + [target_col])
        df_test = df_test.dropna(subset=feature_cols + [target_col])

        if df_train.empty or df_test.empty:
            print(f"      ⚠️ Fold {fold+1} ignorado (dados insuficientes).")
            continue

        X_train, y_train = df_train[feature_cols].fillna(0).values, df_train[target_col].values
        X_test, y_test = df_test[feature_cols].fillna(0).values, df_test[target_col].values

        print(f"      Fold {fold+1}: treino={len(X_train)}, teste={len(X_test)} ({len(X_test)/len(df_clean)*100:.1f}%)")

        try:
            # Escalonamento
            scaler_X = StandardScaler()
            scaler_y = StandardScaler()

            X_train_scaled = scaler_X.fit_transform(X_train)
            X_test_scaled = scaler_X.transform(X_test)
            y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()

            # ✅ GRID SEARCH NOS MODELOS BASE
            base_models = get_optimized_base_models(X_train_scaled, y_train_scaled, n_jobs=-1)

            # ✅ GRID SEARCH NO META-MODELO
            stacking = get_optimized_meta_model(X_train_scaled, y_train_scaled, base_models)

            # Medir tempo de predição
            start_time = time.time()
            y_pred_scaled = stacking.predict(X_test_scaled)
            prediction_time = time.time() - start_time
            
            y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

            metrics = calculate_metrics(y_test, y_pred, prediction_time)

            if metrics['rmse'] is not None:
                all_metrics.append(metrics)
                all_prediction_times.append(prediction_time)
                print(f"        ✅ RMSE: {metrics['rmse']:.2f}M, R²: {metrics['r2']:.4f}, "
                      f"Tempo: {metrics['prediction_time_per_sample']:.4f}ms/amostra")

        except Exception as e:
            print(f"        ⚠️ Erro no fold {fold+1}: {e}")
            continue

    # Cálculo da média das métricas
    if all_metrics:
        avg_metrics = {}
        for metric in ['rmse', 'nrmse', 'r2', 'mape', 'prediction_time_per_sample']:
            values = [m[metric] for m in all_metrics if m[metric] is not None]
            avg_metrics[metric] = round(np.mean(values), 4) if values else None
        
        if all_prediction_times:
            avg_metrics['total_prediction_time'] = round(np.mean(all_prediction_times), 4)
            avg_metrics['total_samples_predicted'] = sum(len(m) for m in [
                df_clean.iloc[test_idx] for _, test_idx in tscv.split(df_clean)
            ] if len(test_idx) <= int(len(df_clean) * test_fraction))

        print(f"    📊 Média - RMSE: {avg_metrics['rmse']:.2f}M, R²: {avg_metrics['r2']:.4f}, "
              f"Tempo: {avg_metrics['prediction_time_per_sample']:.4f}ms/amostra")
        return avg_metrics
    else:
        print(f"    ❌ Nenhuma métrica válida calculada")
        return {"rmse": None, "nrmse": None, "r2": None, "mape": None, "prediction_time_per_sample": None}

# ========== PIPELINE COMPLETO DE AVALIAÇÃO ==========

def evaluate_file(file_path: Path, missing_fractions: List[float]) -> Dict:
    """
    Pipeline completo de avaliação para um arquivo CSV.
    """
    print(f"\n{'='*80}")
    print(f"[AVALIANDO] {file_path.name}")
    print(f"{'='*80}")
    
    df = pd.read_csv(file_path)
    source = file_path.stem.replace("_merged", "").replace("_largest_subseries", "")
    target_col = "Vazao_BBR"
    
    df_clean = get_clean_data(df, target_col)
    
    if len(df_clean) < 20:
        print(f"  ⚠️ Dados insuficientes: apenas {len(df_clean)} amostras")
        return None
    
    results = {}
    
    for frac in missing_fractions:
        print(f"\n  {'─'*60}")
        print(f"  FRAÇÃO DE MISSING: {frac:.0%}")
        print(f"  {'─'*60}")
        
        baseline_results = evaluate_baselines(df_clean, frac, target_col)
        stacking_results = evaluate_stacking_cv(df_clean, frac, target_col)
        
        results[str(frac)] = {
            "baseline": baseline_results,
            "stacking": {
                "mean": {
                    "StackingRegressor": stacking_results
                }
            }
        }
    
    return {
        "source": source,
        "results": results,
        "n_samples": len(df_clean)
    }

# ========== ANÁLISE DE DESEMPENHO ==========

def analyze_stacking_performance(results: Dict) -> Dict[str, Any]:
    """
    Analisa se o stacking foi melhor que as baselines.
    """
    stacking_wins = 0
    total_comparisons = 0
    stacking_rmse_list = []
    best_baseline_rmse_list = []
    prediction_times = []
    
    for frac, data in results.items():
        baseline_data = data.get("baseline", {})
        stacking_data = data.get("stacking", {}).get("mean", {}).get("StackingRegressor", {})
        
        if not baseline_data or not stacking_data:
            continue
        
        stacking_rmse = stacking_data.get("rmse")
        if stacking_rmse is None:
            continue
        
        pred_time = stacking_data.get("prediction_time_per_sample")
        if pred_time is not None:
            prediction_times.append(pred_time)
        
        baseline_rmses = [
            metrics["rmse"] for metrics in baseline_data.values() 
            if metrics.get("rmse") is not None
        ]
        
        if not baseline_rmses:
            continue
        
        best_baseline_rmse = min(baseline_rmses)
        
        total_comparisons += 1
        stacking_rmse_list.append(stacking_rmse)
        best_baseline_rmse_list.append(best_baseline_rmse)
        
        if stacking_rmse < best_baseline_rmse:
            stacking_wins += 1
    
    if total_comparisons == 0:
        return {
            "should_impute": False,
            "win_rate": 0.0,
            "avg_improvement": 0.0,
            "total_comparisons": 0,
            "avg_prediction_time_per_sample": None
        }
    
    win_rate = stacking_wins / total_comparisons
    avg_stacking_rmse = np.mean(stacking_rmse_list)
    avg_best_baseline_rmse = np.mean(best_baseline_rmse_list)
    avg_improvement = ((avg_best_baseline_rmse - avg_stacking_rmse) / avg_best_baseline_rmse) * 100
    
    avg_pred_time = round(np.mean(prediction_times), 4) if prediction_times else None
    
    return {
        "should_impute": win_rate >= 0.5,
        "win_rate": win_rate,
        "avg_improvement": avg_improvement,
        "total_comparisons": total_comparisons,
        "stacking_wins": stacking_wins,
        "avg_stacking_rmse": avg_stacking_rmse,
        "avg_baseline_rmse": avg_best_baseline_rmse,
        "avg_prediction_time_per_sample": avg_pred_time
    }

# ========== IMPUTAÇÃO DE DADOS ==========

def impute_with_baselines(df: pd.DataFrame, target_col: str, output_dir: Path, source: str):
    """
    Imputa valores -1 usando métodos baseline.
    """
    mask_missing = df[target_col] == -1
    n_missing = mask_missing.sum()
    
    if n_missing == 0:
        return
    
    print(f"    [BASELINES] Imputando {n_missing} valores...")
    
    df_clean = df[df[target_col] != -1].copy()
    cols_to_save = ["Data", "Atraso(ms)", "Hop_count", "Bottleneck", target_col, "is_imputed"]
    
    # MÉDIA
    try:
        df_mean = df.copy()
        df_mean['is_imputed'] = 0
        df_mean.loc[mask_missing, 'is_imputed'] = 1
        df_mean.loc[mask_missing, target_col] = df_clean[target_col].mean()
        df_mean[cols_to_save].to_csv(output_dir / f"{source}_baseline_mean.csv", index=False)
        print(f"      ✓ Mean: {output_dir / f'{source}_baseline_mean.csv'}")
    except Exception as e:
        print(f"      ✗ Mean falhou: {e}")
    
    # MEDIANA
    try:
        df_median = df.copy()
        df_median['is_imputed'] = 0
        df_median.loc[mask_missing, 'is_imputed'] = 1
        df_median.loc[mask_missing, target_col] = df_clean[target_col].median()
        df_median[cols_to_save].to_csv(output_dir / f"{source}_baseline_median.csv", index=False)
        print(f"      ✓ Median: {output_dir / f'{source}_baseline_median.csv'}")
    except Exception as e:
        print(f"      ✗ Median falhou: {e}")
    
    # KNN
    try:
        df_knn = df.copy()
        df_knn['is_imputed'] = 0
        df_knn.loc[mask_missing, 'is_imputed'] = 1
        imputer = KNNImputer(n_neighbors=min(5, len(df_clean) // 2))
        df_knn[target_col] = imputer.fit_transform(df[[target_col]].replace(-1, np.nan)).ravel()
        df_knn[cols_to_save].to_csv(output_dir / f"{source}_baseline_knn.csv", index=False)
        print(f"      ✓ KNN: {output_dir / f'{source}_baseline_knn.csv'}")
    except Exception as e:
        print(f"      ✗ KNN falhou: {e}")
    
    # FORWARD FILL
    try:
        df_ffill = df.copy()
        df_ffill['is_imputed'] = 0
        df_ffill.loc[mask_missing, 'is_imputed'] = 1
        df_ffill[target_col] = df_ffill[target_col].replace(-1, np.nan).ffill().bfill()
        df_ffill[cols_to_save].to_csv(output_dir / f"{source}_baseline_ffill.csv", index=False)
        print(f"      ✓ ForwardFill: {output_dir / f'{source}_baseline_ffill.csv'}")
    except Exception as e:
        print(f"      ✗ ForwardFill falhou: {e}")
    
    # BACKWARD FILL
    try:
        df_bfill = df.copy()
        df_bfill['is_imputed'] = 0
        df_bfill.loc[mask_missing, 'is_imputed'] = 1
        df_bfill[target_col] = df_bfill[target_col].replace(-1, np.nan).bfill().ffill()
        df_bfill[cols_to_save].to_csv(output_dir / f"{source}_baseline_bfill.csv", index=False)
        print(f"      ✓ BackwardFill: {output_dir / f'{source}_baseline_bfill.csv'}")
    except Exception as e:
        print(f"      ✗ BackwardFill falhou: {e}")

def impute_with_stacking(df: pd.DataFrame, target_col: str, output_dir: Path, source: str):
    """
    Imputa valores -1 usando Stacking Regressor OTIMIZADO com Grid Search.
    """
    mask_missing = df[target_col] == -1
    n_missing = mask_missing.sum()
    
    if n_missing == 0:
        return
    
    print(f"    [STACKING OTIMIZADO] Imputando {n_missing} valores...")
    
    try:
        df_clean = df[df[target_col] != -1].copy()
        
        if len(df_clean) < 10:
            print(f"      ✗ Dados insuficientes para treinar stacking")
            return
        
        df_clean_feat = engineer_features_for_imputation(df_clean, target_col)
        
        exclude_cols = {target_col, 'Data', 'mask_applied'}
        feature_cols = [c for c in df_clean_feat.columns 
                       if c not in exclude_cols and pd.api.types.is_numeric_dtype(df_clean_feat[c])]
        
        X_train = df_clean_feat[feature_cols].fillna(0).values
        y_train = df_clean_feat[target_col].values
        
        print(f"      Treinando com {len(X_train)} amostras e {len(feature_cols)} features")
        
        # Escalonamento
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X_train_scaled = scaler_X.fit_transform(X_train)
        y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
        
        # ✅ GRID SEARCH NOS MODELOS BASE
        print(f"      🔍 Otimizando modelos base...")
        base_models = get_optimized_base_models(X_train_scaled, y_train_scaled, n_jobs=-1)
        
        # ✅ GRID SEARCH NO META-MODELO
        print(f"      🔍 Otimizando meta-modelo...")
        stacking = get_optimized_meta_model(X_train_scaled, y_train_scaled, base_models)
        
        # IMPUTAR LINHA POR LINHA
        df_imputed = df.copy()
        df_imputed['is_imputed'] = 0
        
        missing_indices = df[mask_missing].index.tolist()
        imputed_values = []
        
        print(f"      Imputando {len(missing_indices)} valores...")
        
        for idx in missing_indices:
            df_temp = df_imputed.iloc[:idx+1].copy()
            
            if df_temp.loc[idx, target_col] == -1:
                valid_values = df_temp[df_temp[target_col] != -1][target_col]
                if len(valid_values) > 0:
                    df_temp.loc[idx, target_col] = valid_values.median()
                else:
                    df_temp.loc[idx, target_col] = df_clean[target_col].median()
            
            df_temp_feat = engineer_features_for_imputation(df_temp, target_col)
            X_pred = df_temp_feat.iloc[-1:][feature_cols].fillna(0).values
            X_pred_scaled = scaler_X.transform(X_pred)
            
            y_pred_scaled = stacking.predict(X_pred_scaled)
            y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()[0]
            
            df_imputed.loc[idx, target_col] = y_pred
            df_imputed.loc[idx, 'is_imputed'] = 1
            imputed_values.append(y_pred)
        
        # Salvar resultado
        cols_to_save = ["Data", "Atraso(ms)", "Hop_count", "Bottleneck", target_col, "is_imputed"]
        output_file = output_dir / f"{source}_stacking_optimized.csv"
        df_imputed[cols_to_save].to_csv(output_file, index=False)
        
        # Estatísticas
        imputed_array = np.array(imputed_values)
        print(f"      ✓ Stacking Otimizado: {output_file}")
        print(f"      ✓ Valores imputados: {n_missing}")
        print(f"      ✓ Média: {np.mean(imputed_array)/1e6:.2f}M")
        print(f"      ✓ Desvio: {np.std(imputed_array)/1e6:.2f}M")
        print(f"      ✓ Min: {np.min(imputed_array)/1e6:.2f}M, Max: {np.max(imputed_array)/1e6:.2f}M")
        print(f"      ✓ Valores únicos: {len(np.unique(imputed_array))}/{n_missing}")
        
    except Exception as e:
        print(f"      ✗ Stacking falhou: {e}")
        import traceback
        traceback.print_exc()

def intelligent_imputation(file_path: Path, results: Dict, output_dir: Path):
    """
    Decide se deve imputar baseado nos resultados da avaliação.
    """
    source = results["source"]
    analysis = analyze_stacking_performance(results["results"])
    
    print(f"\n{'='*80}")
    print(f"[ANÁLISE DE DESEMPENHO] {source}")
    print(f"{'='*80}")
    print(f"  Vitórias do Stacking: {analysis['stacking_wins']}/{analysis['total_comparisons']} ({analysis['win_rate']*100:.1f}%)")
    print(f"  RMSE Médio Stacking: {analysis['avg_stacking_rmse']:.2f}M")
    print(f"  RMSE Médio Melhor Baseline: {analysis['avg_baseline_rmse']:.2f}M")
    print(f"  Melhoria Média: {analysis['avg_improvement']:.2f}%")
    
    if analysis.get('avg_prediction_time_per_sample') is not None:
        print(f"  ⏱️  Tempo Médio de Predição: {analysis['avg_prediction_time_per_sample']:.4f}ms/amostra")
    
    df = pd.read_csv(file_path)
    target_col = "Vazao_BBR"
    n_missing = (df[target_col] == -1).sum()
    
    if n_missing == 0:
        print(f"  ℹ️  Nenhum valor faltante para imputar")
        return
    
    print(f"  📊 {n_missing} valores faltantes encontrados")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    if analysis['should_impute']:
        print(f"  ✅ DECISÃO: Stacking melhor - IMPUTANDO COM TODOS OS MÉTODOS")
        print(f"\n  {'─'*60}")
        
        impute_with_baselines(df, target_col, output_dir, source)
        impute_with_stacking(df, target_col, output_dir, source)
        
        print(f"  {'─'*60}")
        print(f"  ✅ Imputação concluída para {source}")
        
    else:
        print(f"  ❌ DECISÃO: Stacking NÃO foi melhor - APENAS BASELINES")
        print(f"\n  {'─'*60}")
        
        impute_with_baselines(df, target_col, output_dir, source)
        
        print(f"  {'─'*60}")
        print(f"  ⚠️  Stacking não aplicado para {source}")

# ========== EXECUÇÃO PRINCIPAL ==========

def main():
    """
    Executa avaliação completa e imputação inteligente com Grid Search.
    """
    data_path = Path("../../datasets/multivariada-post-process")
    results_path = Path("../../results")
    imputed_path = Path("../../datasets/multivariada-imputed-optimized")
    
    results_path.mkdir(exist_ok=True, parents=True)
    imputed_path.mkdir(exist_ok=True, parents=True)
    
    csv_files = list(data_path.glob("*_merged.csv"))
    missing_fractions = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
    
    print(f"\n{'='*80}")
    print(f"PIPELINE OTIMIZADO COM GRID SEARCH")
    print(f"{'='*80}")
    print(f"Arquivos encontrados: {len(csv_files)}")
    print(f"Frações de missing: {missing_fractions}")
    print(f"Pasta de resultados: {results_path}")
    print(f"Pasta de dados imputados: {imputed_path}")
    print(f"{'='*80}\n")
    
    all_results = {}
    summary = {
        "total_files": len(csv_files),
        "processed": 0,
        "stacking_used": 0,
        "baseline_only": 0,
        "failed": 0
    }
    
    for i, file_path in enumerate(csv_files, 1):
        print(f"\n{'#'*80}")
        print(f"[{i}/{len(csv_files)}] Processando: {file_path.name}")
        print(f"{'#'*80}")
        
        try:
            print(f"\n{'='*80}")
            print(f"FASE 1: AVALIAÇÃO COM GRID SEARCH")
            print(f"{'='*80}")
            
            result = evaluate_file(file_path, missing_fractions)
            
            if result is None:
                print(f"  ⚠️ Arquivo ignorado (dados insuficientes)")
                summary["failed"] += 1
                continue
            
            source = result["source"]
            all_results[source] = result["results"]
            
            evaluation_file = results_path / "metrics_summary_optimized.json"
            with open(evaluation_file, 'w') as f:
                json.dump(all_results, f, indent=4)
            
            print(f"\n  💾 Avaliação salva em: {evaluation_file}")
            
            print(f"\n{'='*80}")
            print(f"FASE 2: IMPUTAÇÃO INTELIGENTE")
            print(f"{'='*80}")
            
            intelligent_imputation(file_path, result, imputed_path)
            
            analysis = analyze_stacking_performance(result["results"])
            summary["processed"] += 1
            if analysis['should_impute']:
                summary["stacking_used"] += 1
            else:
                summary["baseline_only"] += 1
            
            print(f"\n  ✅ Concluído: {source}")
            
        except Exception as e:
            print(f"\n  ❌ Erro processando {file_path.name}: {e}")
            import traceback
            traceback.print_exc()
            summary["failed"] += 1
            continue
    
    print(f"\n{'='*80}")
    print(f"RELATÓRIO FINAL")
    print(f"{'='*80}")
    print(f"Total de arquivos: {summary['total_files']}")
    print(f"Processados com sucesso: {summary['processed']}")
    print(f"  ├─ Stacking usado: {summary['stacking_used']} ({summary['stacking_used']/max(1,summary['processed'])*100:.1f}%)")
    print(f"  └─ Apenas baselines: {summary['baseline_only']} ({summary['baseline_only']/max(1,summary['processed'])*100:.1f}%)")
    print(f"Falhas: {summary['failed']}")
    print(f"\n📁 Resultados salvos em:")
    print(f"  ├─ Avaliação: {results_path / 'metrics_summary_optimized.json'}")
    print(f"  └─ Dados imputados: {imputed_path}")
    print(f"{'='*80}\n")
    
    summary_file = results_path / "imputation_summary_optimized.json"
    with open(summary_file, 'w') as f:
        json.dump(summary, f, indent=4)
    
    print(f"📊 Sumário salvo em: {summary_file}\n")

if __name__ == "__main__":
    main()


PIPELINE OTIMIZADO COM GRID SEARCH
Arquivos encontrados: 615
Frações de missing: [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
Pasta de resultados: ..\..\results
Pasta de dados imputados: ..\..\datasets\multivariada-imputed-optimized


################################################################################
[1/615] Processando: ac-am_merged.csv
################################################################################

FASE 1: AVALIAÇÃO COM GRID SEARCH

[AVALIANDO] ac-am_merged.csv
  [DADOS LIMPOS] 1326 amostras válidas (removidos 885 com -1)

  ────────────────────────────────────────────────────────────
  FRAÇÃO DE MISSING: 20%
  ────────────────────────────────────────────────────────────
  [BASELINE] Avaliando fração 20%
    Máscara aplicada: 265/1326 amostras (20%)
  [STACKING] Avaliando com test_fraction=20%
    Cross-validation com 5 splits
      Fold 1: treino=221, teste=221 (16.7%)
      🔍 Iniciando Grid Search nos modelos base...
        - XGBoost...
          ✓ Melho

KeyboardInterrupt: 